In [1]:
import os
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel

In [2]:
lang_pair = 'de_en'
gold_alignment = list()
with open('bucc2017/de-en/de-en.sample.gold', 'r') as in_file:
    for line in in_file.readlines(): 
        src_tag, tgt_tag = line.strip().split('\t')
        gold_alignment.append((src_tag, tgt_tag)) 
        

src2sent = dict()
with open('bucc2017/de-en/de-en.sample.de', 'r') as src_in:
    for line in src_in.readlines():
        src_tag, src_sent = line.strip().split('\t')
        src2sent[src_tag] = src_sent
        
tgt2sent = dict()
with open('bucc2017/de-en/de-en.sample.en', 'r') as tgt_in:
    for line in tgt_in.readlines():
        tgt_tag, tgt_sent = line.strip().split('\t')
        tgt2sent[tgt_tag] = tgt_sent
        
    
        
src_list = []
tgt_list = []
for src_tag, tgt_tag in gold_alignment:
    src_list.append(src2sent[src_tag])
    tgt_list.append(tgt2sent[tgt_tag])

In [3]:
bert_path = 'bert-base-multilingual-cased'
tokenizer = BertTokenizer.from_pretrained(bert_path, do_lower_case=False)

bert_model = TFBertModel.from_pretrained(bert_path, output_hidden_states=True)

In [ ]:
src_input = tokenizer.batch_encode_plus(src_list, pad_to_max_length=True)
src_input_ids = tf.constant(src_input['input_ids'])
src_attn_masks =  tf.constant(src_input['attention_mask'])

src_data = tf.data.Dataset.from_tensor_slices((src_input_ids, src_attn_masks))

emb_list = []
for input_batch, attn_batch in src_data.batch(20):
    _, _, src_embs = bert_model(input_batch, attention_mask=attn_batch)
    src_embs = tf.reduce_mean(src_embs[7], axis=1)
    emb_list.append(src_embs)
    
src_embeddings = tf.stack(emb_list, axis=0)